# Methods Paper Imagery Plots

In [1]:
%matplotlib widget
import os
os.environ["GDAL_DATA"] = "/Users/parndt/anaconda3/envs/eeicelakes-env/share/gdal"
os.environ["PROJ_LIB"] = "/Users/parndt/anaconda3/envs/eeicelakes-env/share/proj"
os.environ["PROJ_DATA"] = "/Users/parndt/anaconda3/envs/eeicelakes-env/share/proj"
import ee
import h5py
import math
import datetime
import requests
import traceback
import shapely
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import timezone
from datetime import timedelta
from datetime import datetime 
import rasterio as rio
from rasterio import plot as rioplot
from rasterio import warp
import matplotlib
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
from cmcrameri import cm as cmc
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image, display

from lakeanalysis.utils import dictobj, convert_time_to_string, read_melt_lake_h5

In [2]:
# out_path_csv = '/Volumes/nox/Philipp/IceLakesRun1/GlacierLakeDetectionICESat2/GLD1_all_lakes.csv'
# out_path_csv = '/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/GLD2_lakestats_for-wais_ross-fix_gooddata.csv'
# # out_path_csv = '/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/GLD2_all_returned_lakes.csv'
# df = pd.read_csv(out_path_csv)
# df_lakes = df.copy()

In [3]:
# cols = cmc.batlow(np.linspace(0,1,10))
# print('palette: [', end='')
# for i,c in enumerate(cols):
#     cl = np.round(c[:3]*255.0).astype(np.uint8)
#     print("'#{:02x}{:02x}{:02x}'".format(*cl), end='')
#     if i < (len(cols)-1):
#         print(', ', end='')
# print('],')

In [4]:
viz_bands = ['B4', 'B3', 'B2']
viz_bands_rename = ['R', 'G', 'B']
min_sun_elevation = 20

#####################################################################
def get_cloudfree_image_collection(area_of_interest, date_time, days_buffer, max_cloud_scene=50, max_cloud_mask=20):
    
    datetime_requested = datetime.strptime(date_time, '%Y-%m-%dT%H:%M:%SZ')
    start_date = (datetime_requested - timedelta(days=days_buffer)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_date = (datetime_requested + timedelta(days=days_buffer)).strftime('%Y-%m-%dT%H:%M:%SZ')
    # print('Looking for images from %s to %s' % (start_date, end_date), end=' ')
    
    def get_sentinel2_collection(area_of_interest, start_date, end_date):
        sentinel2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                                .filterBounds(area_of_interest)
                                .filterDate(start_date, end_date)
                                .filterMetadata('MEAN_SOLAR_ZENITH_ANGLE', 'less_than', ee.Number(90).subtract(min_sun_elevation)))
    
        s2cloudless_collection = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                                  .filterBounds(area_of_interest)
                                  .filterDate(start_date, end_date))
    
        return (ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
            'primary': sentinel2_collection,
            'secondary': s2cloudless_collection,
            'condition': ee.Filter.equals(**{
                'leftField': 'system:index',
                'rightField': 'system:index'
            })
        })))
    
    def add_cloud_bands_S2(image):
        cloud = ee.Image(image.get('s2cloudless')).select('probability').rename('cloudScore')
        scene_id = image.get('PRODUCT_ID')
        return image.addBands(cloud).set('scene_id', scene_id)
    
    def mask_S2(img):
        mask = img.select('cloudScore').lt(max_cloud_mask)
        return img.updateMask(mask)
    
    def normalize_brightness_S2(image):
        return image.addBands(image.select(viz_bands).rename(viz_bands_rename))
    
    def get_landsat_collection(area_of_interest, start_date, end_date):
        L8T1 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
        L8T2 = ee.ImageCollection('LANDSAT/LC08/C02/T2_TOA')
        L9T1 = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA')
        L9T2 = ee.ImageCollection('LANDSAT/LC09/C02/T2_TOA')
        return (L8T1.merge(L8T2).merge(L9T2).merge(L9T2)
                .filterBounds(area_of_interest)
                .filterDate(start_date, end_date)
                   .filterMetadata('SUN_ELEVATION', 'greater_than', min_sun_elevation))
    
    def landsat_cloud_score(image):
        cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud').rename('cloudScore')
        scene_id = image.get('LANDSAT_PRODUCT_ID')
        return image.addBands(cloud).set('scene_id', scene_id)
    
    def mask_landsat(img):
        mask = img.select('cloudScore').lt(max_cloud_mask)
        return img.updateMask(mask)
    
    def normalize_brightness_LST(image):
        return image.addBands(image.select(viz_bands).rename(viz_bands_rename))
    
    def get_sentinel2_cloud_collection(area_of_interest, start_date, end_date, max_cloud_scene=50, max_cloud_mask=20):
        
        def set_cloudiness(img, aoi=area_of_interest):
            cloudprob = img.select(['cloudScore']).reduceRegion(reducer=ee.Reducer.mean(), 
                                                                 geometry=aoi, 
                                                                 bestEffort=True, 
                                                                 maxPixels=1e6)
            return img.set('ground_track_cloud_prob', cloudprob.get('cloudScore'))
        
        return (get_sentinel2_collection(area_of_interest, start_date, end_date)
                         .map(add_cloud_bands_S2)
                         .map(set_cloudiness)
                         # .filter(ee.Filter.lt('ground_track_cloud_prob', max_cloud_scene))
                         .map(mask_S2)
                         .map(normalize_brightness_S2)
               )
    
    def get_landsat_cloud_collection(area_of_interest, start_date, end_date, max_cloud_scene=50, max_cloud_mask=20):
        
        def set_cloudiness(img, aoi=area_of_interest):
            cloudprob = img.select(['cloudScore']).reduceRegion(reducer=ee.Reducer.mean(), 
                                                                 geometry=aoi, 
                                                                 bestEffort=True, 
                                                                 maxPixels=1e6)
            return img.set('ground_track_cloud_prob', cloudprob.get('cloudScore'))
        
        return (get_landsat_collection(area_of_interest, start_date, end_date)
                         .map(landsat_cloud_score)
                         .map(set_cloudiness)
                         # .filter(ee.Filter.lt('ground_track_cloud_prob', max_cloud_scene))
                         .map(mask_landsat)
                         .map(normalize_brightness_LST)
               )
    
    # def clipToROI(img):
    #     return img.clip(area_of_interest)

    S2_collection = get_sentinel2_cloud_collection(area_of_interest, start_date, end_date, max_cloud_scene, max_cloud_mask)
    LST_collection = get_landsat_cloud_collection(area_of_interest, start_date, end_date, max_cloud_scene, max_cloud_mask)

    return S2_collection.merge(LST_collection)

#####################################################################
def download_imagery(fn, lk, gt, imagery_filename, days_buffer=3, max_cloud_scene=50, max_cloud_mask=20, gamma_value=1.8, 
                     buffer_factor=1.2, crs_out='EPSG:3031', n_imgs_mosaic=7, scale_out=10, mosaic_method='mosaic'):

    lake_mean_delta_time = lk.mframe_data.dt.mean()
    ATLAS_SDP_epoch_datetime = datetime(2018, 1, 1, tzinfo=timezone.utc) # 2018-01-01:T00.00.00.000000 UTC, from ATL03 data dictionary 
    ATLAS_SDP_epoch_timestamp = datetime.timestamp(ATLAS_SDP_epoch_datetime)
    lake_mean_timestamp = ATLAS_SDP_epoch_timestamp + lake_mean_delta_time
    lake_mean_datetime = datetime.fromtimestamp(lake_mean_timestamp, tz=timezone.utc)
    time_format_out = '%Y-%m-%dT%H:%M:%SZ'
    is2time = datetime.strftime(lake_mean_datetime, time_format_out)

    # get the bounding box
    lon_rng = gt.lon.max() - gt.lon.min()
    lat_rng = gt.lat.max() - gt.lat.min()
    fac = 0.25
    bbox = [gt.lon.min()-fac*lon_rng, gt.lat.min()-fac*lat_rng, gt.lon.max()+fac*lon_rng, gt.lat.max()+fac*lat_rng]
    poly = [(bbox[x[0]], bbox[x[1]]) for x in [(0,1), (2,1), (2,3), (0,3), (0,1)]]
    roi = ee.Geometry.Polygon(poly)

    # get the earth engine collection
    collection_size = 0
    if days_buffer > 200:
        days_buffer = 200
    increment_days = days_buffer
    while (collection_size<n_imgs_mosaic) & (days_buffer <= 200):

        cloudfree_collection = get_cloudfree_image_collection(area_of_interest=roi, 
                                                    date_time=is2time, 
                                                    days_buffer=days_buffer, 
                                                    max_cloud_scene=max_cloud_scene, 
                                                    max_cloud_mask=max_cloud_mask)

        cloudfree_collection = cloudfree_collection.filter(ee.Filter.lt('ground_track_cloud_prob', max_cloud_scene))
        collection_size = cloudfree_collection.size().getInfo()
        days_buffer += increment_days
    
    # get the time difference between ICESat-2 and Sentinel-2 and sort by it 
    # is2time = lk.date_time
                         
    def set_time_difference(img, is2time=lake_mean_timestamp):
        timediff = ee.Date(lake_mean_timestamp*1000).difference(img.get('system:time_start'), 'second').abs()
        return img.set('timediff', timediff)
    cloudfree_collection = cloudfree_collection.map(set_time_difference).sort('timediff')

    # create a region around the ground track over which to download data
    lon_center = gt.lon.mean()
    lat_center = gt.lat.mean()
    gt_length = gt.x10.max() - gt.x10.min()
    point_of_interest = ee.Geometry.Point(lon_center, lat_center)
    region_of_interest = point_of_interest.buffer(np.nanmax((gt_length*0.5*buffer_factor,500)))

    prod_id = 'no imagery'
    if collection_size > 0:
        
        # stretch the color values 
        def color_stretch(image):
            percentiles = image.select(viz_bands_rename).reduceRegion(**{
                'reducer': ee.Reducer.percentile(**{'percentiles': [3, 97], 'outputNames': ['lower', 'upper']}),
                'geometry': region_of_interest,
                'scale': 30,
                'maxPixels': 1e9,
                'bestEffort': True
            })
            lower = percentiles.select(['.*_lower']).values().reduce(ee.Reducer.min())
            upper = percentiles.select(['.*_upper']).values().reduce(ee.Reducer.max())
            return image.select(viz_bands_rename).unitScale(lower, upper).clamp(0,1).resample('bilinear').reproject(**{'crs': crs_out,'scale': scale_out})

        # select the first image for info
        # info = cloudfree_collection.getInfo()
        # for f in info['features']:
        #     print(f['properties']['scene_id'])
        #     print(f['properties']['timediff'])
        selectedImage = cloudfree_collection.first()
        
        # limit mosaic to first n images after sorting by timediff
        to_mosaic = cloudfree_collection.limit(n_imgs_mosaic).map(color_stretch)
        
        if mosaic_method == 'mean':
            rgb = to_mosaic.mean()
        elif mosaic_method == 'median':
            rgb = to_mosaic.median()
        else:
            # mosaic() puts last image in collection on top, so sort by timediff in descending order (the False value)
            rgb = to_mosaic.sort('timediff', False).mosaic()
            
        rgb_gamma = rgb.pow(1/gamma_value)
        rgb8bit = rgb_gamma.clamp(0,1).multiply(255).uint8()
        
        # from the selected image get some stats: product id, cloud probability and time difference from icesat-2
        prod_id = selectedImage.get('scene_id').getInfo()
        cld_prb = selectedImage.get('ground_track_cloud_prob').getInfo()
        s2_timestamp = selectedImage.get('system:time_start').getInfo()
        s2datetime = datetime.fromtimestamp(s2_timestamp/1000, tz=timezone.utc)
        s2datestr = datetime.strftime(s2datetime, 'time_format_out')
        is2datetime = lake_mean_datetime
        timediff = s2datetime - is2datetime
        timediff_str = '%s' % (timediff)
        
        # get the download URL and download the selected image
        success = False
        tries = 0
        while (success == False) & (tries <= 10):
            try:
                downloadURL = rgb8bit.getDownloadUrl({'name': 'mySatelliteImage',
                                                          'crs': crs_out,
                                                          'scale': scale_out,
                                                          'region': region_of_interest,
                                                          'filePerBand': False,
                                                          'format': 'GEO_TIFF'})
        
                response = requests.get(downloadURL)
                with open(imagery_filename, 'wb') as f:
                    f.write(response.content)
        
                # print('--> Downloaded the 8-bit RGB image as %s.' % imagery_filename)
                success = True
                tries += 1
            except:
                traceback.print_exc()
                scale *= 2
                # print('-> download unsuccessful, increasing scale to %.1f...' % scale)
                success = False
                tries += 1

    return prod_id, timediff_str
            
#####################################################################
def plot_imagery(fn, days_buffer=5, max_cloud_scene=50, max_cloud_mask=20, xlm=[None, None], ylm=[None, None], gamma_value=1.8, imagery_filename=None,
                 re_download=True, ax=None, buffer_factor=1.5, crs_out='EPSG:3031', n_imgs_mosaic=7, scale_out=10, mosaic_method='mosaic'):
                     
    lk = dictobj(read_melt_lake_h5(fn))
    time_format_out = '%Y-%m-%dT%H:%M:%SZ'
    lake_mean_delta_time = lk.mframe_data.dt.mean()
    ATLAS_SDP_epoch_datetime = datetime(2018, 1, 1, tzinfo=timezone.utc) # 2018-01-01:T00.00.00.000000 UTC, from ATL03 data dictionary 
    ATLAS_SDP_epoch_timestamp = datetime.timestamp(ATLAS_SDP_epoch_datetime)
    lake_mean_timestamp = ATLAS_SDP_epoch_timestamp + lake_mean_delta_time
    lake_mean_datetime = datetime.fromtimestamp(lake_mean_timestamp, tz=timezone.utc)
    lake_mean_time_string = datetime.strftime(lake_mean_datetime, time_format_out)
    lk.date_time = lake_mean_time_string
                     
    df = lk.photon_data.copy()
    if not xlm[0]:
        xlm[0] = df.xatc.min()
    if not xlm[1]:
        xlm[1] = df.xatc.max()
    if not ylm[0]:
        ylm[0] = lk.surface_elevation-2*lk.max_depth
    if not ylm[1]:
        ylm[1] = lk.surface_elevation+lk.max_depth
    if not imagery_filename:
        imagery_filename = 'imagery' + fn[fn.rfind('/'):].replace('.h5','.tif')
    
    
    df = df[(df.xatc >= xlm[0]) & (df.xatc <= xlm[1]) & (df.h >= ylm[0]) & (df.h <= ylm[1])].reset_index(drop=True).copy()
    x_off = np.min(df.xatc)
    df.xatc -= x_off
    
    dfd = lk.depth_data.copy()
    dfd.xatc -= x_off

    # get the ground track
    df['x10'] = np.round(df.xatc, -1)
    gt = df.groupby(by='x10')[['lat', 'lon']].median().reset_index()
    lon_center = gt.lon.mean()
    lat_center = gt.lat.mean()
                     
    try:
        prod_id = 'no imagery'
        if ((not os.path.isfile(imagery_filename)) or re_download) and ('modis' not in imagery_filename):
            prod_id, dt_str = download_imagery(fn=fn, lk=lk, gt=gt, imagery_filename=imagery_filename, days_buffer=days_buffer, 
                             max_cloud_scene=max_cloud_scene, max_cloud_mask=max_cloud_mask, gamma_value=gamma_value, 
                             buffer_factor=buffer_factor, crs_out=crs_out, n_imgs_mosaic=n_imgs_mosaic, scale_out=scale_out,
                             mosaic_method=mosaic_method)
        
        try:
            myImage = rio.open(imagery_filename)
            
            # make the figure
            if not ax:
                fig, ax = plt.subplots(figsize=[6,6])
            
            rioplot.show(myImage, ax=ax)
            ax.axis('off')
        except:
            traceback.print_exc()
    
        try:
            dt_str = dt_str.split('.')[0]
            if dt_str[0] != '-':
                dt_str = '+' + dt_str
            text = 'imagery time difference: %s\n%s' % (dt_str, prod_id)
            ax.text(0.01, 0.01, text, fontsize=6, ha='left', va='bottom', transform=ax.transAxes,
               bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.1,rounding_size=0.3', lw=0), zorder=3000)
        except:
            traceback.print_exc()
    
        try:
            ximg, yimg = warp.transform(src_crs='epsg:4326', dst_crs=myImage.crs, xs=np.array(gt.lon), ys=np.array(gt.lat))
            if 'modis' in imagery_filename:
                xrng = ximg[-1] - ximg[0]
                yrng = yimg[-1] - yimg[0]
                fac = 30
                # print('using saved modis image')
                ax.plot([ximg[-1]+fac*xrng,ximg[0]-fac*xrng], [yimg[-1]+fac*yrng, yimg[0]-fac*yrng], 'k:', lw=1)
                ax.annotate('', xy=(ximg[-1]+fac*xrng, yimg[-1]+fac*yrng), xytext=(ximg[0]-fac*xrng, yimg[0]-fac*yrng),
                                 arrowprops=dict(width=0, lw=0, headwidth=5, headlength=5, color='k'),zorder=1000)
                # ax.plot(ximg, yimg, 'r-', lw=1, zorder=5000)
            else:
                # print('plotting ground track')
                ax.annotate('', xy=(ximg[-1], yimg[-1]), xytext=(ximg[0], yimg[0]),
                                 arrowprops=dict(width=0.7, headwidth=5, headlength=5, color='k'),zorder=1000)
                try:
                    isdepth = dfd.depth>0
                    bed = dfd.h_fit_bed
                    bed[~isdepth] = np.nan
                    bed[(dfd.depth>2) & (dfd.conf < 0.3)] = np.nan
                    surf = np.ones_like(dfd.xatc) * lk.surface_elevation
                    surf[~isdepth] = np.nan
                    xatc_surf = np.array(dfd.xatc)[~np.isnan(surf)]
                    lon_bed = np.array(dfd.lon)
                    lat_bed = np.array(dfd.lat)
                    lon_bed[(np.isnan(surf)) & (np.isnan(bed))] = np.nan
                    lat_bed[(np.isnan(surf)) & (np.isnan(bed))] = np.nan
                    xb, yb = warp.transform(src_crs='epsg:4326', dst_crs=myImage.crs, xs=lon_bed, ys=lat_bed)
                    ax.plot(xb, yb, 'r-', lw=1, zorder=5000)
                except:
                    traceback.print_exc()
        except:
            traceback.print_exc()
            
        if not ax:
            fig.tight_layout(pad=0)
    
        return myImage, lon_center, lat_center
    except: 
        return None, lon_center, lat_center
        traceback.print_exc()

                     
#####################################################################
def plotIS2(fn, ax=None, xlm=[None, None], ylm=[None,None], cmap=cmc.lapaz_r, name='ICESat-2 data'):
    lk = dictobj(read_melt_lake_h5(fn))
    time_format_out = '%Y-%m-%dT%H:%M:%SZ'
    lake_mean_delta_time = lk.mframe_data.dt.mean()
    ATLAS_SDP_epoch_datetime = datetime(2018, 1, 1, tzinfo=timezone.utc) # 2018-01-01:T00.00.00.000000 UTC, from ATL03 data dictionary 
    ATLAS_SDP_epoch_timestamp = datetime.timestamp(ATLAS_SDP_epoch_datetime)
    lake_mean_timestamp = ATLAS_SDP_epoch_timestamp + lake_mean_delta_time
    lake_mean_datetime = datetime.fromtimestamp(lake_mean_timestamp, tz=timezone.utc)
    lake_mean_time_string = datetime.strftime(lake_mean_datetime, time_format_out)
    lk.date_time = lake_mean_time_string
    df = lk.photon_data.copy()
    dfd = lk.depth_data.copy()

    try:
        isdepth = dfd.depth>0
        bed = dfd.h_fit_bed
        bed[~isdepth] = np.nan
        bed[(dfd.depth>2) & (dfd.conf < 0.3)] = np.nan
        surf = np.ones_like(dfd.xatc) * lk.surface_elevation
        surf[~isdepth] = np.nan
        surf_only = surf[~np.isnan(surf)]
        bed_only = bed[(~np.isnan(surf)) & (~np.isnan(bed))]
        xatc_surf = np.array(dfd.xatc)[~np.isnan(surf)]
        xatc_bed = np.array(dfd.xatc)[(~np.isnan(surf)) & (~np.isnan(bed))]
        
        # make the figure
        if not ax:
            fig, ax = plt.subplots(figsize=[8,5])
    
        df['is_afterpulse']= df.prob_afterpulse > np.random.uniform(0,1,len(df))
        if not cmap:
            # ax.scatter(df.xatc, df.h, s=1, c='k')
            ax.scatter(df.xatc[~df.is_afterpulse], df.h[~df.is_afterpulse], s=1, c='k')
        else:
            ax.scatter(df.xatc[~df.is_afterpulse], df.h[~df.is_afterpulse], s=1, c=df.snr, cmap=cmap)
        
        ax.scatter(dfd.xatc[isdepth], dfd.h_fit_bed[isdepth], s=4, color='r', alpha=dfd.conf[isdepth])
        ax.plot(dfd.xatc, dfd.h_fit_bed, color='gray', lw=0.5)
        
        ax.plot(dfd.xatc, bed, color='r', lw=1)
        ax.plot(dfd.xatc, surf, color='C0', lw=1)
    except:
        ax.scatter(df.xatc, df.h, s=1, c='k')
        ax.plot(dfd.xatc, dfd.h_fit_bed, 'r-')
        ax.plot(dfd.xatc, np.ones_like(dfd.xatc) * lk.surface_elevation, 'b-')

    # add the length of surface
    try:
        arr_y = lk.surface_elevation+lk.max_depth*0.25
        x_start = np.min(xatc_surf)
        x_end = np.max(xatc_surf)
        x_mid = (x_end + x_start) / 2
        len_surf_m = np.floor((x_end-x_start)/100)*100
        len_surf_km = len_surf_m/1000
        arr_x1 = x_mid - len_surf_m / 2
        arr_x2 = x_mid + len_surf_m / 2
        ax.annotate('', xy=(arr_x1, arr_y), xytext=(arr_x2, arr_y),
                             arrowprops=dict(width=0.7, headwidth=5, headlength=5, color='C0'),zorder=1000)
        ax.annotate('', xy=(arr_x2, arr_y), xytext=(arr_x1, arr_y),
                             arrowprops=dict(width=0.7, headwidth=5, headlength=5, color='C0'),zorder=1000)
        ax.text(x_mid, arr_y, '%.1f km' % len_surf_km, fontsize=12, ha='center', va='bottom', color='C0', fontweight='bold',
                bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.2,rounding_size=0.5', lw=0))
    except:
        traceback.print_exc()

    # add the max depth
    try:
        if np.sum(np.isnan(bed)) < len(bed):
            y_low = np.nanmin(bed)
            if y_low < (lk.surface_elevation - 2 * lk.max_depth):
                bed[bed < lk.surface_elevation - 2 * lk.max_depth] = np.nan
                if np.sum(np.isnan(bed)) < len(bed):
                    y_low = np.nanmin(bed)
            if np.sum(np.isnan(bed)) < len(bed):
                y_up = lk.surface_elevation
                arr_x = xatc_bed[np.argmin(bed_only)]
                xlm = (df.xatc.min(), df.xatc.max())
                arr_x = xlm[0] - 0.0* (xlm[1] - xlm[0])
                y_len = y_up - y_low
                y_mid = (y_up + y_low) / 2
                arr_len = y_len
                arr_y1 = y_mid + arr_len / 2
                arr_y2 = y_mid - arr_len / 2
                ref_index = 1.33
                dep_round = np.round(y_len / ref_index, 1)
                ax.annotate('', xy=(arr_x, arr_y2), xytext=(arr_x, arr_y1),
                                     arrowprops=dict(width=0.7, headwidth=5, headlength=5, color='r'),zorder=1000)
                ax.annotate('', xy=(arr_x, arr_y1), xytext=(arr_x, arr_y2),
                                     arrowprops=dict(width=0.7, headwidth=5, headlength=5, color='r'),zorder=1000)
                ax.text(arr_x, y_mid, '%.1f m' % dep_round, fontsize=12, ha='right', va='center', color='r', fontweight='bold',
                        bbox=dict(facecolor='white', alpha=0.8, lw=0, boxstyle='round,pad=0.2,rounding_size=0.5'), rotation=90)
    except:
        traceback.print_exc()

    # add the title
    try:
        datestr = datetime.strftime(datetime.strptime(lk.date_time,'%Y-%m-%dT%H:%M:%SZ'), '%d %B %Y %H:%M:%S UTC')
        sheet = lk.ice_sheet
        region = lk.polygon_filename.split('_')[-1].replace('.geojson', '')
        if sheet == 'AIS':
            region = region + ' (%s)' % lk.polygon_filename.split('_')[-2]
        latstr = lk.lat_str[:-1] + '°' + lk.lat_str[-1]
        lonstr = lk.lon_str[:-1] + '°' + lk.lon_str[-1]
        description = '%s, %s - %s (%s, %s, %.1fm)\n%s %s (%s)' % (datestr, sheet, region, latstr, lonstr, lk.surface_elevation, lk.granule_id, lk.gtx, lk.beam_strength)
        
        ax.text(0.5, 1.0, description, fontsize=8, ha='center', va='top', transform=ax.transAxes,
               bbox=dict(facecolor='white', alpha=0.9, boxstyle='round,pad=0.2,rounding_size=0.5', lw=0), zorder=3000)
    except:
        traceback.print_exc()

    try:
        xlm = (df.xatc.min(), df.xatc.max())
        dp = lk.max_depth
        if dp < 2.0:
            dp = 2.0
        ylm = (lk.surface_elevation-dp*2, lk.surface_elevation+dp)
        ax.set_xlim(xlm)
        ax.set_ylim(ylm)
    except:
        traceback.print_exc()
        
    ax.axis('off')

    
#####################################################################
def plot_IS2_imagery(fn, axes=None, xlm=[None,None], ylm=[None,None], cmap=None, days_buffer=5, max_cloud_scene=50, max_cloud_mask=50,
                     gamma_value=1.8, imagery_filename=None, re_download=True, img_aspect=3/2, name='ICESat-2 data',
                     return_fig=False, n_imgs_mosaic=7, scale_out=10, mosaic_method='mosaic'):

    if not axes:
        fig = plt.figure(figsize=[12,6], dpi=80)
        gs = fig.add_gridspec(1,3)
        axp = [fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1:])]
    else:
        axp = axes
        
    ax = axp[1]
    try:
        plotIS2(fn=fn, ax=ax, xlm=xlm, ylm=ylm, cmap=cmap, name=name)
    except:
        ax.text(0.5, 0.5, 'plotting error', fontsize=12, ha='center', va='center', transform=ax.transAxes)
        traceback.print_exc()
    
    ax = axp[0]
    try:
        crs_out = 'EPSG:3413' if '_GrIS_' in fn else 'EPSG:3031'
        center_lon, center_lat = 0, 0
        img, center_lon, center_lat = plot_imagery(fn=fn, days_buffer=days_buffer, max_cloud_scene=max_cloud_scene, 
            max_cloud_mask=max_cloud_mask, xlm=xlm, ylm=ylm, gamma_value=gamma_value, imagery_filename=imagery_filename, 
            re_download=re_download, ax=ax, crs_out=crs_out, n_imgs_mosaic=n_imgs_mosaic, scale_out=scale_out,
            mosaic_method=mosaic_method)
        
        if img:        
            if imagery_filename:
                if 'modis' in imagery_filename:
                    center_x, center_y = warp.transform(src_crs='epsg:4326', dst_crs=img.crs, xs=[center_lon], ys=[center_lat])
                    center_x = center_x[0]
                    center_y = center_y[0]
                    rng = 220000
                    if img_aspect > 1:
                        ax.set_xlim(center_x - 0.5*rng/img_aspect, center_x + 0.5*rng/img_aspect)
                        ax.set_ylim(center_y - 0.5*rng, center_y + 0.5*rng)
                    if img_aspect < 1:
                        ax.set_xlim(center_x - 0.5*rng, center_x + 0.5*rng)
                        ax.set_ylim(center_y - 0.5*rng*img_aspect, center_y + 0.5*rng*img_aspect)
                    
            elif (img_aspect > 1): 
                h_rng = img.bounds.top - img.bounds.bottom
                cntr = (img.bounds.right + img.bounds.left) / 2
                ax.set_xlim(cntr-0.5*h_rng/img_aspect, cntr+0.5*h_rng/img_aspect)
            elif img_aspect < 1: 
                w_rng = img.bounds.right - img.bounds.left
                cntr = (img.bounds.top + img.bounds.bottom) / 2
                ax.set_ylim(cntr-0.5*w_rng*img_aspect, cntr+0.5*w_rng/img_aspect)
    except:
        ax.text(0.5, 0.5, 'plotting error', fontsize=12, ha='center', va='center', transform=ax.transAxes)
        traceback.print_exc()
            
    
    if not axes:
        fig.tight_layout(pad=1, h_pad=0, w_pad=0)
        if not name:
            name = 'zzz' + lk.polygon_filename.split('_')[-1].replace('.geojson', '')
        outname = 'figplots/' + name.replace(' ', '') + fn[fn.rfind('/')+1:].replace('.h5','.jpg')
        fig.savefig(outname, dpi=300)

    if return_fig:
        plt.close(fig)
        return center_lon, center_lat, fig
    else:
        return center_lon, center_lat

In [5]:
# base = '/Users/parndt/jupyterprojects/IceLakesRun2/methods_data/'
# searchfor = '.h5'
# filelist = [base+f for f in os.listdir(base) \
#             if os.path.isfile(os.path.join(base, f)) & (searchfor in f)]
# filelist.sort()
# listlength = len(filelist)
# print(listlength)

In [6]:
base = '/Users/parndt/jupyterprojects/IceLakesRun2'
oldfolder = '/upto21jun2023_data/'
searchdir = base + oldfolder
searchfor = '.h5'
filelist = [searchdir+f for f in os.listdir(searchdir) \
            if os.path.isfile(os.path.join(searchdir, f)) & (searchfor in f)]
filelist.sort()
listlength = len(filelist)
print(listlength)

25094


In [7]:
# filelist = [
#     base + 'lake_10000000_AIS_2018-19_simplified_ANT_1000_East_B-C_ATL03_20190128061524_04700212_006_02_gt1r_0006.h5',
#            ]

# for i, fn in enumerate(filelist):
#     display(Image(fn.replace('/methods_data/', '/methods_plots/').replace('.h5', '_imagery.jpg')))

filelist = filelist[:10]
listlength = len(filelist)
print(listlength)

10


In [8]:
##### FOR EVERYTHING UP TO JUN 21, 2023

base = '/Users/parndt/jupyterprojects/IceLakesRun2'
oldfolder = '/upto21jun2023_data/'
searchdir = base + oldfolder
searchfor = '.h5'
filelist = [searchdir+f for f in os.listdir(searchdir) \
            if os.path.isfile(os.path.join(searchdir, f)) & (searchfor in f)]
filelist.sort()
listlength = len(filelist)
print('\nThere are %d lake files to process...' % listlength)

import time
from IPython.display import clear_output
def time_string(secs):
    m, s = divmod(secs, 60)
    h, m = divmod(m, 60)
    d, h = divmod(h, 24)
    return '%d days, %d hrs, %d mins, %d secs' % (d, h, m, s)
plt.close('all')

# startfile = 0
# nfiles = 5
# for i, fn in enumerate(filelist[startfile:startfile+nfiles]):

tstart = time.time()
idone = 0
for i, fn in enumerate(filelist):

    fn_plot = fn.replace(oldfolder, oldfolder.replace('_data/', '_plots/')).replace('.h5', '_imagery.jpg')
    fn_imagery = fn.replace(oldfolder, oldfolder.replace('_data/', '_imagery/')).replace('.h5', '_imagery.tif')

    if not os.path.isfile(fn_plot):
        
        tnow = time.time()
        selapsed = tnow - tstart
        elapsed = time_string(selapsed)
        if idone > 0:
            sremaining = (listlength-i) / idone * selapsed
            remaining = time_string(sremaining)
        else:
            remaining = 'undefined'
        ns = int(np.round((i+1)/listlength*100))
        print('\n[' + '#'*ns + '.'*(100-ns) + ']\n')
        print('%i / %i lakes processed (%i this run)' % (i+1, listlength, idone))
        print('%s elapsed' % elapsed)
        print('%s remaining' % remaining)
        print('current file: %s' % fn_plot.split('/')[-1])
        if i < (listlength-1):
            clear_output(wait=True)

        idone += 1

        settings = {
            're_download': False,     # True
            'img_aspect': 1.0,        # 1.0
            'days_buffer': 5,         # 5
            'max_cloud_scene': 20,    # 20               (make this smaller, more like 30?)
            'max_cloud_mask': 40,     # 50               (make this smaller, more like 30?)
            'n_imgs_mosaic': 7,       # 7                (put to one for checking code)
            'mosaic_method': 'mean',  # mean             (alternatively "median" or "mosaic")
            'scale_out': 10,          # 10
            'gamma_value': 1.0,       # 1.0
            'xlm': [None, None],      # [None, None]     (keep at None unless adjusting for a particular plot)
            'ylm': [None, None],      # [None, None]     (keep at None unless adjusting for a particular plot)
            'return_fig': False       # False
        }
    
        fig = plt.figure(figsize=[10,4.35])
        gs = fig.add_gridspec(ncols=9, nrows=1)
        
        axs = []
        axs.append(fig.add_subplot(gs[0, :4])) 
        axs.append(fig.add_subplot(gs[0, 4:]))

        try:
            lk = dictobj(read_melt_lake_h5(fn))
            
            plot_IS2_imagery(fn=fn, imagery_filename=fn_imagery, **settings, axes=axs)
            fig.tight_layout(pad=0.3, h_pad=0.3, w_pad=0.4)
        except:
            axs[0].text(0.5, 0.5, 'plotting error', fontsize=12, ha='center', va='center', transform=axs[0].transAxes)
            traceback.print_exc()
    
        plt.close(fig)
        fig.savefig(fn_plot, dpi=600)

    # display(Image(fn_plot))

print('\n\nscp -i ~/.ssh/lumosKey parndt@132.239.169.168:%s /mnt/c/Users/phili/Documents/IceLakesMethods/gooddata/' % fn_plot)


[####################################################################################################]

25094 / 25094 lakes processed (1877 this run)
0 days, 4 hrs, 16 mins, 42 secs elapsed
0 days, 0 hrs, 0 mins, 8 secs remaining
current file: lake_10000000_GrIS_2023_simplified_GRE_2000_SW_ATL03_20230618212225_13671905_006_01_gt3r_0002_imagery.jpg


scp -i ~/.ssh/lumosKey parndt@132.239.169.168:/Users/parndt/jupyterprojects/IceLakesRun2/upto21jun2023_plots/lake_10000000_GrIS_2023_simplified_GRE_2000_SW_ATL03_20230618212225_13671905_006_01_gt3r_0002_imagery.jpg /mnt/c/Users/phili/Documents/IceLakesMethods/gooddata/


In [12]:
! ls /Users/parndt/jupyterprojects/IceLakesRun2/upto21jun2023_data | wc -l

   50188


In [13]:
! ls /Users/parndt/jupyterprojects/IceLakesRun2/upto21jun2023_plots | wc -l

   25094


In [ ]:
##### FOR METHODS PAPER

base = '/Users/parndt/jupyterprojects/IceLakesRun2/methods_data/'
searchfor = '.h5'
filelist = [base+f for f in os.listdir(base) \
            if os.path.isfile(os.path.join(base, f)) & (searchfor in f)]
filelist.sort()
listlength = len(filelist)
print(listlength)

plt.close('all')

# startfile = 0
# nfiles = 5
# for i, fn in enumerate(filelist[startfile:startfile+nfiles]):

for i, fn in enumerate(filelist):

    fn_plot = fn.replace('/methods_data/', '/methods_plots/').replace('.h5', '_imagery.jpg')
    fn_imagery = fn.replace('/methods_data/', '/methods_imagery/').replace('.h5', '_imagery.tif')

    print('%5i / %5i : %s                      ' % (i+1, listlength, fn_plot.split('/')[-1]), end='\r')

    settings = {
        're_download': True,      # True
        'img_aspect': 1.0,        # 1.0
        'days_buffer': 5,         # 5
        'max_cloud_scene': 20,    # 20               (make this smaller, more like 30?)
        'max_cloud_mask': 40,     # 50               (make this smaller, more like 30?)
        'n_imgs_mosaic': 7,       # 7                (put to one for checking code)
        'mosaic_method': 'mean',  # mean             (alternatively "median" or "mosaic")
        'scale_out': 10,          # 10
        'gamma_value': 1.0,       # 1.0
        'xlm': [None, None],      # [None, None]     (keep at None unless adjusting for a particular plot)
        'ylm': [None, None],      # [None, None]     (keep at None unless adjusting for a particular plot)
        'return_fig': False       # False
    }

    fig = plt.figure(figsize=[10,4.35])
    gs = fig.add_gridspec(ncols=9, nrows=1)
    
    axs = []
    axs.append(fig.add_subplot(gs[0, :4])) 
    axs.append(fig.add_subplot(gs[0, 4:]))
    
    try:
        lk = dictobj(read_melt_lake_h5(fn))
        
        plot_IS2_imagery(fn=fn, imagery_filename=fn_imagery, **settings, axes=axs)
        fig.tight_layout(pad=0.3, h_pad=0.3, w_pad=0.4)
    except:
        axs[0].text(0.5, 0.5, 'plotting error', fontsize=12, ha='center', va='center', transform=axs[0].transAxes)
        traceback.print_exc()

    # plt.close(fig)
    fig.savefig(fn_plot, dpi=600)

    # display(Image(fn_plot))

print('\n\nscp -i ~/.ssh/lumosKey parndt@132.239.169.168:%s /mnt/c/Users/phili/Documents/IceLakesMethods/gooddata/' % fn_plot)

In [ ]:
# poi = ee.Geometry.Point(lk.lon, lk.lat)
# roi = poi.buffer(1000)
# col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(roi).filterDate('2019-06-01', '2019-06-30')
# info = col.getInfo()
# info = col.getInfo()
# for f in info['features']:
#     print(f['properties']['system:time_start'])
print('scp -i ~/.ssh/lumosKey parndt@132.239.169.168:%s /mnt/c/Users/phili/Documents/IceLakesMethods/gooddata/' % fn_plot)

In [ ]:
time_format_out = '%Y-%m-%dT%H:%M:%SZ'
lake_mean_time_string = datetime.strftime(lake_mean_datetime, time_format_out)
base = '/Users/parndt/jupyterprojects/IceLakesRun2/methods_data/'
fn = base + 'lake_09022414_GrIS_2019_simplified_GRE_2000_CW_ATL03_20190617064249_12220303_006_02_gt2l_0017.h5'
lk = dictobj(read_melt_lake_h5(fn))

image = ee.Image('LANDSAT/LC08/C02/T1/LC08_007012_20190617')
image_timestamp = image.get('system:time_start').getInfo()
image_datetime = datetime.fromtimestamp(image_timestamp/1000, tz=timezone.utc)
image_time_string = datetime.strftime(image_datetime, time_format_out)
image_center_time = image.get('SCENE_CENTER_TIME').getInfo()
image_date_acquired = image.get('DATE_ACQUIRED').getInfo()

time_format_out = '%Y-%m-%dT%H:%M:%SZ'
lake_mean_delta_time = lk.mframe_data.dt.mean()
ATLAS_SDP_epoch_datetime = datetime(2018, 1, 1, tzinfo=timezone.utc) # 2018-01-01:T00.00.00.000000 UTC, from ATL03 data dictionary 
ATLAS_SDP_epoch_timestamp = datetime.timestamp(ATLAS_SDP_epoch_datetime)
lake_mean_timestamp = ATLAS_SDP_epoch_timestamp + lake_mean_delta_time
lake_mean_datetime = datetime.fromtimestamp(lake_mean_timestamp, tz=timezone.utc)
lake_mean_time_string = datetime.strftime(lake_mean_datetime, time_format_out)

print('lake time calculated here:   ', lake_mean_time_string)
print('lake time from saved file:   ', lk.date_time)
print('')
print('image center time:           ', image_center_time)
print('image date acquired:         ', image_date_acquired)
print('image time from earth engine:', image_time_string)

timediff = image_datetime - lake_mean_datetime
timediff

In [ ]:
timediff = ee.Date(lake_mean_timestamp*1000).difference(image.get('system:time_start'), 'second').abs()

In [ ]:
timediff.getInfo()

In [ ]:
ee.Date(lake_mean_timestamp*1000).getInfo()

In [ ]:
image.get('system:time_start').getInfo()

In [ ]:
datetime.fromtimestamp(0, tzinfo=timezone.utc)

In [ ]:
from datetime import timezone

In [ ]:
datetime.timestamp(datetime(1970, 1, 1, 0, 0, tzinfo=timezone.utc))

In [ ]:
is2datetime = datetime.strptime('2020-05-11T13:04:24Z', '%Y-%m-%dT%H:%M:%SZ')
s2datetime = datetime.strptime('2020-04-11T15:23:58Z', '%Y-%m-%dT%H:%M:%SZ')
timediff = s2datetime - is2datetime

In [ ]:
'%s' % (timediff)

In [ ]:
ATL03_20190810040312_06580403_006_02_gt3l

In [ ]:
lk.granule_id, lk.gtx, lk.beam_strength

In [ ]:

plt.close('all')

settings = {
    're_download': True,
    'img_aspect': 1.0,
    'days_buffer': 5,
    'gamma_value': 1.0
}

fig = plt.figure(figsize=[10,4.35])
gs = fig.add_gridspec(ncols=9, nrows=1)

axs = []
axs.append(fig.add_subplot(gs[0, :4])) 
axs.append(fig.add_subplot(gs[0, 4:]))

toplot = amery
# toplot = usulluup_sermia
# toplot = ice_cover
# toplot = shackleton
# toplot = tucker
# toplot = more_typical
lk = dictobj(read_melt_lake_h5(toplot['fn']))
plot_IS2_imagery(**toplot, **settings, axes=axs)
fig.tight_layout(pad=0.3, h_pad=0.3, w_pad=0.4)

fig.savefig('plots/example_xxx.jpg', dpi=600)

In [ ]:
base = '/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/detection_out_data/'
base = '/Users/parndt/jupyterprojects/IceLakesRun2/detection_out_data/'

amery = {
    'name': 'Amery Ice Shelf',
    'fn': base + 'lake_09661801_AIS_2021-22_simplified_ANT_1000_East_B-C_ATL03_20220118022134_04091412_006_01_gt3l_0006.h5',
    'gamma_value': 0.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

shackleton = {
    'name': 'Shackleton Ice Shelf',
    'fn': base + 'lake_09996401_AIS_2019-20_simplified_ANT_1000_East_C-Cp_ATL03_20200224204246_09180610_006_01_gt2r_0002.h5',
    'gamma_value': 0.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

tucker = {
    'name': 'Tucker Glacier',
    'fn': base + 'lake_09993680_AIS_2020-21_simplified_ANT_1000_East_Dp-E_ATL03_20210128124234_05421012_006_01_gt3r_0001.h5',
    'gamma_value': 0.75,
    'xlm': [None, None],
    'ylm': [None, None],
}

nansen = {
    'name': 'Nansen Ice Shelf',
    'fn': base + 'lake_09999825_AIS_2020-21_simplified_ANT_1000_East_Dp-E_ATL03_20210106141527_02071012_006_01_gt3l_0014.h5',
    'gamma_value': 0.7,
    'xlm': [None, None],
    'ylm': [None, None],
}

koettlitz = {
    'name': 'Koettlitz Glacier',
    'fn': base + 'lake_09999778_AIS_2021-22_simplified_ANT_1000_East_E-Ep_ATL03_20220108204642_02681412_006_01_gt2l_0005.h5',
    'gamma_value': 0.8,
    'xlm': [None, None],
    'ylm': [None, None],
}

byrd = {
    'name': 'Byrd Glacier',
    'fn': base + 'lake_09962662_AIS_2020-21_simplified_ANT_1000_East_E-Ep_ATL03_20210111010129_02751011_006_01_gt2l_0002.h5',
    'gamma_value': 0.8,
    'xlm': [None, None],
    'ylm': [None, None],
}

nimrod = {
    'name': 'Nimrod Glacier',
    'fn': base + 'lake_09998718_AIS_2021-22_simplified_ANT_1000_East_E-Ep_ATL03_20220127202459_05581411_006_01_gt3l_0000.h5',
    'gamma_value': 0.8,
    'xlm': [None, None],
    'ylm': [None, None],
}

scott_other = {
    'name': 'Scott Glacier',
    'fn': base + 'lake_09479207_AIS_2021-22_test_ross_ATL03_20220104191501_02061411_006_01_gt3l_0000.h5',
    'imagery_filename': 'imagery/modis_scott_2022-01-04.tif',
    'xlm': [None, None],
    'ylm': [None, None],
}

scott = {
    'name': 'Scott Glacier',
    'fn': base + 'lake_09999381_AIS_2018-19_simplified_ANT_1000_West_Ep-F_ATL03_20190313202056_11510211_006_02_gt3l_0003.h5',
    'imagery_filename': 'imagery/modis_mercer-scott_2019-01-02.tif',
    'xlm': [None, None],
    'ylm': [None, None],
}

mercer = {
    'name': 'Mercer Ice Stream',
    'fn': base + 'lake_09998311_AIS_2018-19_simplified_ANT_1000_West_Ep-F_ATL03_20190102075052_00740211_006_02_gt1l_0000.h5',
    'imagery_filename': 'imagery/modis_mercer-scott_2019-01-02.tif',
    'xlm': [None, None],
    'ylm': [None, None],
}

pineisland = {
    'name': 'Pine Island',
    'fn': base + 'lake_09915690_AIS_2019-20_simplified_ANT_1000_West_G-H_ATL03_20191224015224_13460512_006_01_gt2l_0000.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

wilkins = {
    'name': 'Wilkins Ice Shelf',
    'fn': base + 'lake_09941497_AIS_2022-23_simplified_ANT_1000_Peninsula_Hp-I_ATL03_20230206171923_07351812_006_01_gt2r_0002.h5',
    'gamma_value': 0.75,
    'xlm': [30, 900],
    'ylm': [None, None],
}

bach = {
    'name': 'Bach Ice Shelf',
    'fn': base + 'lake_09650303_AIS_2019-20_simplified_ANT_1000_Peninsula_Hp-I_ATL03_20200118094903_03460610_006_01_gt3r_0025.h5',
    'gamma_value': 0.75,
    'xlm': [None, None],
    'ylm': [None, None],
}

georgevi = {
    'name': 'George VI Ice Shelf',
    'fn': base + 'lake_09809632_AIS_2022-23_simplified_ANT_1000_Peninsula_Hp-I_ATL03_20230129173611_06131812_006_01_gt3r_0080.h5',
    'gamma_value': 0.65,
    'xlm': [95, None],
    'ylm': [None, None],
}

larsenc = {
    'name': 'Larsen C Ice Shelf',
    'fn': base + 'lake_09927824_AIS_2021-22_simplified_ANT_1000_Peninsula_I-Ipp_ATL03_20220208212635_07421410_006_01_gt2l_0000.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

filchner = {
    'name': 'Filchner Ice Shelf',
    'fn': base + 'lake_09999779_AIS_2018-19_simplified_ANT_1000_East_Jpp-K_ATL03_20190217224511_07860211_006_02_gt1l_0000.h5',
    'gamma_value': 1.0,
    'max_cloud_prob': 30,
    'xlm': [None, None],
    'ylm': [None, None],
}

riiser_larsen = {
    'name': 'Riiser-Larsen Ice Shelf',
    'fn': base + 'lake_09964391_AIS_2019-20_simplified_ANT_1000_East_K-A_ATL03_20200128053234_04960610_006_01_gt2r_0001.h5',
    'gamma_value': 0.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

riiser_larsen2 = {
    'name': 'Riiser-Larsen Ice Shelf',
    'fn': base + 'lake_09999986_AIS_2019-20_simplified_ANT_1000_East_K-A_ATL03_20200210044152_06940610_006_01_gt3r_0003.h5',
    'gamma_value': 0.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

fimbul = {
    'name': 'Fimbul Ice Shelf',
    'fn': base + 'lake_09971131_AIS_2019-20_simplified_ANT_1000_East_A-Ap_ATL03_20200115044852_02970610_006_01_gt1r_0001.h5',
    'gamma_value': 0.75,
    'xlm': [None, None],
    'ylm': [None, None],
}

nivil = {
    'name': 'Nivil Ice Shelf',
    'fn': base + 'lake_07269598_AIS_2019-20_simplified_ANT_1000_East_A-Ap_ATL03_20200116042313_03120610_006_01_gt2r_0002.h5',
    'gamma_value': 0.6,
    'xlm': [None, None],
    'ylm': [None, None],
}

roi_baudouin = {
    'name': 'Roi Baudouin Ice Shelf',
    'fn': base + 'lake_09999544_AIS_2019-20_simplified_ANT_1000_East_A-Ap_ATL03_20200201150322_05630612_006_01_gt2l_0005.h5',
    'gamma_value': 0.6,
    'xlm': [None, None],
    'ylm': [None, None],
}

roi_baudouin2 = {
    'name': 'Roi Baudouin Ice Shelf',
    'fn': base + 'lake_09999966_AIS_2019-20_simplified_ANT_1000_East_A-Ap_ATL03_20200123025757_04180610_006_01_gt2r_0003.h5',
    'gamma_value': 0.6,
    'xlm': [None, None],
    'ylm': [None, None],
}

#######################################################
# GREENLAND
usulluup_sermia = { # SW, 1
    'name': 'Usulluup Sermia',
    'fn': base + 'lake_05857276_GrIS_2019_simplified_GRE_2000_SW_ATL03_20190818034635_07800403_006_02_gt1l_0000.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

jakobshavn_isbrae = { # CW, 2
    'name': 'Jakobshavn Isbrae',
    'fn': base + 'lake_07661472_GrIS_2019_simplified_GRE_2000_CW_ATL03_20190716051841_02770403_006_02_gt3l_0014.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

alison_gletscher = { # NW, 3
    'name': 'Alison Gletscher',
    'fn': base + 'lake_05549443_GrIS_2019_simplified_GRE_2000_NW_ATL03_20190805043712_05820403_006_02_gt1l_0003.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

humboldt_gletscher = { # NO, 4
    'name': 'Humboldt Gletscher',
    'fn': base + 'lake_08835579_GrIS_2021_simplified_GRE_2000_NO_ATL03_20210726182948_05061203_006_01_gt2r_0002.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

storstrommen = { # NE, 5
    'name': 'Storstrommen',
    'fn': base + 'lake_02254624_GrIS_2021_simplified_GRE_2000_NE_ATL03_20210720053125_04061205_006_01_gt3r_0042.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

kangerlussuaq = { # CE, 6
    'name': 'Kangerlussuaq',
    'fn': base + 'lake_09456922_GrIS_2022_simplified_GRE_2000_CE_ATL03_20220815111248_08331605_006_01_gt1r_0002.h5',
    'gamma_value': 1.4,
    'xlm': [None, None],
    'ylm': [None, None],
}

helheimgletscher = { # SE, 7
    'name': 'Helheim Gletscher',
    'fn': base + 'lake_09906186_GrIS_2021_simplified_GRE_2000_SE_ATL03_20210622185325_13741103_006_01_gt1r_0001.h5',
    'gamma_value': 1.1,
    'xlm': [None, None],
    'ylm': [None, None],
}

ryder_gletscher = { # NO, 8 (one of the deepest good-looking lakes)
    'name': 'Ryder Gletscher',
    'fn': base + 'lake_09803461_GrIS_2019_simplified_GRE_2000_NO_ATL03_20190902161841_10170404_006_02_gt1l_0000.h5',
    'gamma_value': 1.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

nioghalvfjerdsfjorden = { # NO, 9 (one of the longest good-looking lakes)
    'name': 'Nioghalvfjerdsfjorden',
    'fn': base + 'lake_09130391_GrIS_2020_simplified_GRE_2000_NE_ATL03_20200802222701_05890805_006_01_gt2l_0004.h5',
    'gamma_value': 1.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

sermeq_kangaasarsuup = { # SW, 10 
    'name': 'Sermeq Kangaasarsuup',
    'fn': base + 'lake_08989585_GrIS_2020_simplified_GRE_2000_SW_ATL03_20200721114126_03990803_006_01_gt1l_0002.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

harald_moltke_brae = { # NW, 11
    'name': 'Harald Moltke Brae',
    'fn': base + 'lake_09019555_GrIS_2019_simplified_GRE_2000_NW_ATL03_20190730053649_04910403_006_02_gt2l_0004.h5',
    'gamma_value': 1.3,
    'xlm': [None, None],
    'ylm': [None, None],
}

eielson_hare_fjord = { # CE, 12
    'name': 'Eielson Gletscher',
    'fn': base + 'lake_09916382_GrIS_2021_simplified_GRE_2000_CE_ATL03_20210728051446_05281205_006_01_gt2r_0002.h5',
    'gamma_value': 1.3,
    'xlm': [None, None],
    'ylm': [None, None],
}

academy = { # NO, 13
    'name': 'Academy Glacier',
    'fn': base + 'lake_09687033_GrIS_2022_simplified_GRE_2000_NO_ATL03_20220822111641_09401604_006_01_gt2r_0003.h5',
    'gamma_value': 1.3,
    'xlm': [None, None],
    'ylm': [None, None],
}

sermeq_silarleq = { # CW, 13
    'name': 'Sermeq Silarleq',
    'fn': base + 'lake_08819453_GrIS_2019_simplified_GRE_2000_CW_ATL03_20190818034635_07800403_006_02_gt1l_0001.h5',
    'gamma_value': 1.3,
    'xlm': [None, None],
    'ylm': [None, None],
}

highest_quality = {
    'name': '"Highest Quality" ICESat-2 Lake',
    'fn': base + 'lake_02254624_GrIS_2021_simplified_GRE_2000_NE_ATL03_20210720053125_04061205_006_01_gt3r_0042.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

deepest = {
    'name': 'Deepest ICESat-2 Lake',
    'fn': base + 'lake_09803461_GrIS_2019_simplified_GRE_2000_NO_ATL03_20190902161841_10170404_006_02_gt1l_0000.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

deepest2 = {
    'name': 'Deepest ICESat-2 Lake',
    'fn': base + 'lake_09974979_GrIS_2019_simplified_GRE_2000_NE_ATL03_20190803021956_05500403_006_02_gt2l_0004.h5',
    'gamma_value': 1.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

longest = { 
    'name': 'Longest ICESat-2 Lake',
    'fn': base + 'lake_09130391_GrIS_2020_simplified_GRE_2000_NE_ATL03_20200802222701_05890805_006_01_gt2l_0004.h5',
    'gamma_value': 1.5,
    'xlm': [None, None],
    'ylm': [None, None],
}

multiple_basins = {
    'name': 'ICESat-2 Lake with Multiple Basins',
    'fn': base + 'lake_09999207_AIS_2019-20_simplified_ANT_1000_East_B-C_ATL03_20200207122923_06530612_006_01_gt3r_0020.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

more_typical = {
    'name': 'a more typical ICESat-2 lake...',
    'fn': base + 'lake_09999996_GrIS_2019_simplified_GRE_2000_CW_ATL03_20190712052659_02160403_006_02_gt2l_0007.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

shelf_bench = {
    'name': 'What is this weird lake?',
    'fn': base + 'lake_09928802_AIS_2020-21_simplified_ANT_1000_East_C-Cp_ATL03_20201118081610_08420910_006_01_gt2l_0000.h5',
    'gamma_value': 1.0,
    'xlm': [None, 880],
    'ylm': [None, None],
}

ice_cover = {
    'name': 'An ICESat-2 Lake With Ice Cover',
    'fn': base + 'lake_09769063_GrIS_2019_simplified_GRE_2000_NW_ATL03_20190907030457_10850403_006_02_gt2r_0005.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

afterpulses = {
    'name': 'Afterpulses Removed',
    'fn': base + 'lake_09586908_GrIS_2022_simplified_GRE_2000_CW_ATL03_20220714010847_03381603_006_02_gt2r_0015.h5',
    'gamma_value': 1.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

doline_dammed = {
    'name': 'ICESat-2 Lake Dammed by Doline',
    'fn': base + 'lake_09991923_AIS_2018-19_simplified_ANT_1000_East_B-C_ATL03_20190123173555_04010210_006_02_gt2l_0003.h5',
    'gamma_value': 0.3,
    'xlm': [None, None],
    'ylm': [None, None],
}

longest_ant = {
    'name': 'Longest ICESat-2 Lake',
    'fn': base + 'lake_09992056_AIS_2018-19_simplified_ANT_1000_East_B-C_ATL03_20190123173555_04010210_006_02_gt2r_0008.h5',
    'gamma_value': 0.3,
    'xlm': [None, None],
    'ylm': [None, None],
}

furthest_south = {
    'name': 'Furthest South',
    'fn': base + 'lake_09998311_AIS_2018-19_simplified_ANT_1000_West_Ep-F_ATL03_20190102075052_00740211_006_02_gt1l_0000.h5',
    'imagery_filename': 'imagery/modis_mercer-scott_2019-01-02.tif',
    'xlm': [None, None],
    'ylm': [None, None],
}

rift_foot = {
    'name': 'Rift with bottom return?',
    'fn': base + 'lake_09994291_AIS_2020-21_simplified_ANT_1000_West_F-G_ATL03_20201125223347_09580910_006_01_gt2l_0000.h5',
    'gamma_value': 2.0,
    'xlm': [None, None],
    'ylm': [None, None],
}

In [ ]:
plt.close('all')

settings = {
    're_download': True,
    'img_aspect': 1.0,
    'days_buffer': 5,
}

fig = plt.figure(figsize=[10,4.5])
gs = fig.add_gridspec(ncols=3, nrows=1)

axs = []
axs.append(fig.add_subplot(gs[0, 0])) 
axs.append(fig.add_subplot(gs[0, 1:]))

toplot = amery
# toplot = usulluup_sermia
# toplot = ice_cover
# toplot = shackleton
# toplot = tucker
# toplot = more_typical
lk = dictobj(read_melt_lake_h5(toplot['fn']))
plot_IS2_imagery(**toplot, **settings, axes=axs)
fig.tight_layout(pad=0.3, h_pad=0.3, w_pad=0.4)

fig.savefig('plots/example_xxx.jpg', dpi=600)

In [ ]:
print(lk.lat, lk.lon, lk.date_time, lk.rgt, lk.gtx)

In [ ]:
filelst = [
'/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/detection_out_data/lake_09999733_AIS_2019-20_simplified_ANT_1000_Peninsula_Hp-I_ATL03_20200204213749_06130612_006_01_gt3l_0053.h5',
'/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/detection_out_data/lake_09999905_AIS_2022-23_simplified_ANT_1000_Peninsula_Hp-I_ATL03_20230129173611_06131812_006_01_gt1l_0002.h5',
'/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/detection_out_data/lake_09955969_AIS_2020-21_simplified_ANT_1000_Peninsula_Hp-I_ATL03_20210213150425_07881010_006_01_gt1r_0001.h5',
'/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/detection_out_data/lake_09998273_AIS_2019-20_simplified_ANT_1000_Peninsula_Hp-I_ATL03_20200106230147_01710612_006_01_gt3l_0006.h5',
'/Volumes/nox/Philipp/IceLakesRun2/GlacierLakeDetectionICESat2/detection_out_data/lake_09999182_AIS_2022-23_simplified_ANT_1000_Peninsula_I-Ipp_ATL03_20230220160301_09481812_006_01_gt2l_0000.h5',
]

In [ ]:
for fn in filelst:
    settings = {
        're_download': False,
        'img_aspect': 1.4,
        'days_buffer': 30,
        'xlm': [None, None],
        'ylm': [None, None],
        'gamma_value': 1.0,
    }
    plot_IS2_imagery(fn=fn, **settings)

In [ ]:
fig = plt.figure(figsize=[32,16], dpi=30)
gs = fig.add_gridspec(6, 18)
axs = []
axs.append(fig.add_subplot(gs[1:5, 3:6])) # greenland
axs.append(fig.add_subplot(gs[1:5, 9:15])) # antarctica
for j in range(0, 18, 3):
    axs.append(fig.add_subplot(gs[0, j]))
    axs.append(fig.add_subplot(gs[0, j+1:j+3]))
for i in range(1,5):
    for j in np.array([0, 2, 5])*3:
        axs.append(fig.add_subplot(gs[i, j]))
        axs.append(fig.add_subplot(gs[i, j+1:j+3]))
for j in range(0, 18, 3):
    axs.append(fig.add_subplot(gs[5, j]))
    axs.append(fig.add_subplot(gs[5, j+1:j+3]))

ax = axs[0]
gre_bound.plot(color='k', ax=ax, lw=0.5)
gre_gdf_merged.exterior.plot(color='gray', ax=ax, lw=0.3)
ax.axis('off')

ax = axs[1]
ant_gdf_merged.exterior.plot(color='gray', ax=ax, lw=0.3)
ant_gdf_shelf.plot(color='blue', alpha=0.1, ax=ax, lw=0)
ant_bound.plot(color='k', ax=ax, lw=0.5)
ant_gdf_shelf.boundary.plot(color='k', ax=ax, lw=0.5)
ax.axis('off')

for ax in axs:
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])

fig.tight_layout(pad=0, h_pad=0, w_pad=0)

for i, ax in enumerate(axs):
    ax.text(0.5, 0.5, '%i'%i, ha='center', va='center', transform=ax.transAxes, fontsize=20)

fig.tight_layout(pad=1, h_pad=1, w_pad=1)

In [ ]:
base = '/Users/parndt/jupyterprojects/IceLakesRun2/detection_out_data/'
fn = 'lake_09022414_GrIS_2019_simplified_GRE_2000_CW_ATL03_20190617064249_12220303_006_02_gt2l_0017.h5'
lk = dictobj(read_melt_lake_h5(base+fn))

In [ ]:
lake_mean_delta_time = lk.mframe_data.dt.iloc[0]

In [ ]:
lake_mean_delta_time = lk.mframe_data.dt.mean()
ATLAS_SDP_epoch = '2018-01-01:T00.00.00.000000Z'  # UTC, from ATL03 data dictiornary 
ATLAS_SDP_epoch_format = '%Y-%m-%d:T%H.%M.%S.%fZ'  # UTC, from ATL03 data dictiornary
time_format_out = '%Y-%m-%d %H:%M:%S'
ATLAS_SDP_epoch_datetime = datetime.strptime(ATLAS_SDP_epoch, ATLAS_SDP_epoch_format)
ATLAS_SDP_epoch_timestamp = datetime.timestamp(ATLAS_SDP_epoch_datetime)
lake_mean_timestamp = ATLAS_SDP_epoch_timestamp + lake_mean_delta_time
lake_mean_datetime = datetime.fromtimestamp(lake_mean_timestamp)
lake_mean_time_string = datetime.strftime(lake_mean_datetime, time_format_out)
lake_mean_time_string

In [ ]:
imagery_prod_id = 'LC08_L1TP_007012_20190617_20200827_02_T1'

In [ ]:
lk.lat

In [ ]:
lk.lon